In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser

In [ ]:
# Database Setup

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.mars_db
collection = db.articles

## Step 1 - Scraping

## NASA Mars News

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
news_title = soup.find('div', class_='content_title').a.text
news_p = soup.find('div', class_='article_teaser_body').text

## JPL Mars Space Images - Featured Image

In [4]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image_style = soup.find('article')['style']
image_url = image_style.split("'")[1].split("'")[0]

featured_image_url = 'https://www.jpl.nasa.gov' + image_url

## Mars Weather

In [5]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

## Mars Facts


In [6]:
url = 'https://space-facts.com/mars/'

tables = pd.read_html(url)

In [7]:
mars_facts_df = tables[1]
mars_facts_df.columns = ['Description', 'Value']

In [8]:
mars_facts_df.set_index('Description', inplace=True)

In [9]:
html_table = mars_facts_df.to_html()

html_table = html_table.replace('\n', '')

## Mars Hemispheres

In [10]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

hemisphere_image_urls = []

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('div', class_='item')

In [11]:
# Create for loop to get image url and title of all hemispheres

for result in results:
        link = result.find('h3').text
        title = link.replace(' Enhanced', '')
        browser.click_link_by_partial_text(link)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        result2 = soup.find('img', class_='wide-image')
        url = 'https://astrogeology.usgs.gov/' + result2["src"]
        hemisphere_image_urls.append({"title":title, "img_url":url})
        browser.back()

In [13]:
mars_data = {}

mars_data['latest_news_title'] = news_title
mars_data['latest_news_article_p'] = news_p

mars_data['featured_img'] = featured_image_url

mars_data['weather_tweet'] = mars_weather

mars_data['html_table'] = html_table

mars_data['hemisphere_imgs'] = hemisphere_image_urls

In [14]:
mars_data

{'latest_news_title': "NASA's Curiosity Rover Finds an Ancient Oasis on Mars",
 'latest_news_article_p': "New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.",
 'featured_img': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18848-1920x1200.jpg',
 'weather_tweet': 'InSight sol 313 (2019-10-14) low -101.4ºC (-150.6ºF) high -27.2ºC (-16.9ºF)\nwinds from the SSE at 5.5 m/s (12.4 mph) gusting to 22.5 m/s (50.3 mph)\npressure at 7.20 hPapic.twitter.com/2moNAouxXa',
 'html_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moo